교내 정보 딜라이트에서 가져오기

In [21]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import re

In [22]:
url = 'https://delight.duksung.ac.kr/ko/program/all'

In [23]:
# 기본 URL 및 페이지 수 설정
base_url = 'https://delight.duksung.ac.kr/ko/program/all/list/0/'
page_count = 6

In [24]:
# 결과를 저장할 리스트
all_data = []

1) 일단 덩어리 불러오는 함수

In [25]:
def fetch_list_items(base_url, page_count):
    """주어진 URL에서 ul data-role='list' 안의 li 요소를 가져오는 함수"""
    all_items = []

    for page in range(1, page_count + 1):  # 1페이지부터 page_count까지
        # 각 페이지 URL 생성
        url = f'{base_url}{page}'

        # HTTP 요청
        response = requests.get(url)

        # HTML 분석
        soup = BeautifulSoup(response.text, 'html.parser')

        # ul[data-role='list'] 안의 li 요소 찾기
        ul_element = soup.select_one("ul[data-role='list']")
        if ul_element:
            li_elements = ul_element.find_all("li")  # 해당 ul 내의 모든 <li> 요소 선택
            all_items.extend(li_elements)  # 각 <li>를 리스트에 추가

    return all_items

In [26]:
# 함수 호출
default_divs = fetch_list_items(base_url, page_count)
# 결과 출력
print(f"총 {len(default_divs)}개의 'li' div 요소가 발견되었습니다.")

총 72개의 'li' div 요소가 발견되었습니다.


2. 타이틀 불러오기

In [27]:
def extract_titles(div_elements):
    """주어진 div 요소 리스트에서 타이틀을 추출하는 함수"""
    titles = []
    for div in div_elements:
        title_element = div.select_one('b.title')  # <b class="title"> 태그 찾기
        if title_element:
            titles.append(title_element.text.strip())  # 타이틀 추가
    return titles

In [28]:
# 타이틀 추출
titles = extract_titles(default_divs)
# 결과 출력
print(f"총 {len(titles)}개의 타이틀이 발견되었습니다.")
for title in titles:
    print(title)

총 72개의 타이틀이 발견되었습니다.
2024학년도  덕성여대 W쇼핑 영상컨텐츠공모전
2024년도 중견기업 바로알기 취업특강
2024-2 학생참여 공모형 워크숍 주제공모
2024년도 2학기 바이어스
2024학년도  창업아이템 경진대회
[재맞고] 넥스트드림 - 꿈을 이루는 포트폴리오
2024-2 PBL, 창의융합, 캡스톤디자인 현장연계 산학연 교과 학습역량강화교육
[인권센터]2024 찾아가는 재학생 폭력예방교육(2차)
2024학년도 융복합 교양인 아이디어 공모전
2024 커리어 플랫폼 사회진출(4학년) 프로그램
2024학년도 덕성 나다움(I:DS) 스피치 대회 청중모집
2024학년도 덕성 나다움(I:DS) 스피치 대회
2024 커리어플랫폼 1학년 프로그램
2024년도  면접시리즈2(모의면접편)
2024  DUKSUNG 지역협업 아이디어 공모전
[재맞고] 넥스트드림 - 꿈을 이루는 포트폴리오
2024 하반기 온라인 일본기업 채용설명회
2024년도 공기업 길라잡이
2024학년도 생성형 AI 프롬프트대회
[재맞고] 넥스트드림 - 꿈을 이루는 포트폴리오
2024-2 PBL, 창의융합, 캡스톤디자인 현장연계 산학연 교과 학습역량강화교육
글로벌 이슈 카드 뉴스 콘테스트
2024-2 PBL, 창의융합, 캡스톤디자인 현장연계 산학연 교과 학습역량강화교육
[재맞고] 넥스트드림 - 꿈을 이루는 포트폴리오
[재맞고] 넥스트드림 - 꿈을 이루는 포트폴리오
2024-2 PBL, 창의융합, 캡스톤디자인 현장연계 산학연 교과 학습역량강화교육
2024-2 PBL, 창의융합, 캡스톤디자인 현장연계 산학연 교과 학습역량강화교육
아미공마일리지(아미공챌린지 (Army-0 challenge))
[2024-2] 데이터 활용 대학 자율혁신을 위한 정책 공모(DS-데이터톤)
2024-2 PBL, 창의융합, 캡스톤디자인 현장연계 산학연 교과 학습역량강화교육
2024 재학생 생명존중 교육
2024-2 PBL, 창의융합, 캡스톤디자인 현장연계 산학연 교과 학습역량강화교육
2024학년도 2학기  진로길라JOB이

3. 이미지 불러오기

In [29]:
def extract_cover_images(div_elements):
    """주어진 div 요소 리스트에서 cover 이미지 URL을 추출하는 함수"""
    all_images = []
    base_url = "https://delight.duksung.ac.kr/"  # 기본 URL

    for div in div_elements:
        cover_element = div.select_one("div.cover")  # div.cover 찾기
        if cover_element:
            style = cover_element.get('style')  # style 속성 가져오기
            if style:
                # background-image URL 추출
                match = re.search(r'url\((.*?)\)', style)
                if match:
                    image_url = match.group(1).strip(" '\"")  # URL에서 따옴표 제거
                    # URL이 상대경로인 경우 base_url 추가
                    if not image_url.startswith('http'):
                        image_url = base_url + image_url.lstrip('/')  # 맨 앞 '/' 제거
                    all_images.append(image_url)
            # 이미지가 없으면 해당 div를 비우고 넘어감
        else:
            continue  # cover 요소가 없으면 넘어감

    return all_images


In [30]:
# cover 이미지 URL 추출
cover_images = extract_cover_images(default_divs)
# 결과 출력
print(f"총 {len(cover_images)}개의 이미지 URL이 발견되었습니다.")
for image in cover_images:
    print(image)

총 71개의 이미지 URL이 발견되었습니다.
https://delight.duksung.ac.kr/attachment/view/31097/cover.jpg?ts=1725262993
https://delight.duksung.ac.kr/attachment/view/31185/cover.jpg?ts=1727415048
https://delight.duksung.ac.kr/attachment/view/32523/cover.jpg?ts=1727855771
https://delight.duksung.ac.kr/attachment/view/31092/cover.jpg?ts=1725254834
https://delight.duksung.ac.kr/attachment/view/32237/cover.jpg?ts=1727851451
https://delight.duksung.ac.kr/attachment/view/32540/cover.jpg?ts=1727427392
https://delight.duksung.ac.kr/attachment/view/32238/cover.jpg?ts=1727312317
https://delight.duksung.ac.kr/attachment/view/31001/cover.jpg?ts=1726030970
https://delight.duksung.ac.kr/attachment/view/30926/cover.jpg?ts=1728030380
https://delight.duksung.ac.kr/attachment/view/31691/cover.jpg?ts=1727158177
https://delight.duksung.ac.kr/attachment/view/31689/cover.jpg?ts=1727051885
https://delight.duksung.ac.kr/attachment/view/30956/cover.jpg?ts=1728030489
https://delight.duksung.ac.kr/attachment/view/31786/cover.jpg?t

4. 기한 끌고오기

In [31]:
def extract_first_dates(div_elements):
    """주어진 div 요소 리스트에서 각 class='content' 안의 가장 위에 있는 time 정보를 추출하는 함수"""
    dates = []
    
    for div in div_elements:
        content_div = div.select_one("div.content")  # class='content'인 div 찾기
        if content_div:
            first_time_element = content_div.select_one("time")  # 첫 번째 time 요소 찾기
            if first_time_element:
                dates.append(first_time_element.text.strip())  # datetime 텍스트 추가

    return dates

In [32]:
# 날짜 추출
dates = extract_first_dates(default_divs)

# 결과 출력
print(f"총 {len(dates)}개의 날짜가 발견되었습니다.")
for date in dates:
    print(date)

총 72개의 날짜가 발견되었습니다.
2024.09.02(Mon)
2024.10.08(Tue) 18:30
2024.09.26(Thu)
2024.09.01(Sun)
2024.09.09(Mon)
2024.10.17(Thu) 16:30
2024.10.16(Wed) 14:00
2024.10.23(Wed) 17:30
2024.09.09(Mon)
2024.10.22(Tue)
2024.09.23(Mon)
2024.09.23(Mon)
2024.10.22(Tue)
2024.10.25(Fri) 00:00
2024.09.23(Mon)
2024.10.24(Thu) 16:30
2024.10.23(Wed) 18:00
2024.10.28(Mon)
2024.10.30(Wed) 18:00
2024.10.31(Thu) 16:30
2024.10.31(Thu) 14:00
2024.09.30(Mon)
2024.11.05(Tue) 14:00
2024.11.07(Thu) 16:30
2024.11.14(Thu) 16:30
2024.11.21(Thu) 14:00
2024.11.28(Thu) 14:00
2024.03.27(Wed)
2024.09.02(Mon)
2024.12.05(Thu) 14:00
2024.03.21(Thu)
2024.12.12(Thu) 14:00
2024.09.02(Mon)
2024.05.02(Thu)
2024.09.02(Mon)
2024.03.27(Wed)
2024.06.03(Mon)
2024.05.27(Mon)
2024.05.27(Mon)
2024.10.28(Mon) 16:30
2024.10.29(Tue) 16:30
2024.09.30(Mon)
2024.09.27(Fri) 19:00
2024.09.26(Thu) 19:00
2024.09.26(Thu)
2024.09.26(Thu) 16:30
2024.09.26(Thu)
2024.09.25(Wed) 16:30
2024.09.25(Wed)
2024.09.24(Tue) 18:30
2024.09.24(Tue) 18:00
2024.09.24(Tue

5. 주최자

In [33]:
def extract_institution_spans(div_elements):
    """주어진 div 요소 리스트에서 class='content' 안의 span.institution 정보를 추출하는 함수"""
    institutions = []
    
    for div in div_elements:
        content_div = div.select_one("div.content")  # class='content'인 div 찾기
        if content_div:
            span_element = content_div.select_one("span.institution")  # span.institution 요소 찾기
            if span_element:
                institutions.append(span_element.text.strip())  # span의 텍스트 추가

    return institutions



In [34]:
# 각 content div에서 span.institution 추출
institution_spans = extract_institution_spans(default_divs)

# 결과 출력
print(f"총 {len(institution_spans)}개의 기관이 발견되었습니다.")
for institution in institution_spans:
    print(institution)

총 72개의 기관이 발견되었습니다.
창업센터
대학일자리본부
교수학습개발센터
글로벌 융합대학
창업센터
대학일자리본부
산학연구과
부속/부설기관
차미리사교양대학
대학일자리본부
차미리사교양대학
차미리사교양대학
대학일자리본부
대학일자리본부
창업지원단
대학일자리본부
대학일자리본부
대학일자리본부
차미리사교양대학
대학일자리본부
산학연구과
글로벌커뮤니케이션센터
산학연구과
대학일자리본부
대학일자리본부
산학연구과
산학연구과
과학기술대학
교육혁신성과관리센터
산학연구과
학생상담센터
산학연구과
대학일자리본부
부속/부설기관
학생상담센터
학생상담센터
대학일자리본부
대학일자리본부
대학일자리본부
도서관
도서관
교수학습개발센터
교수학습개발센터
교수학습개발센터
교수학습개발센터
도서관
대학일자리본부
도서관
교수학습개발센터
대학일자리본부
도서관
도서관
대학일자리본부
차미리사교양대학
캠퍼스타운조성단
교수학습개발센터
대학일자리본부
대학일자리본부
대학일자리본부
국제교류과
대학일자리본부
대학일자리본부
도서관
글로벌커뮤니케이션센터
도서관
사회복지학전공
교수학습개발센터
글로벌커뮤니케이션센터
대학일자리본부
대학일자리본부
교수학습개발센터
대학일자리본부


json으로 내보내기

In [38]:
# 프로그램 데이터를 리스트로 묶기
programs = [
    {
        'title': title,
        'coverImage': cover_image,
        'date': date,
        'institution': institution,
        'parentCategory_id': 1,  # 기본값 설정
        'subCategory_id': 1       # 기본값 설정
    }
    for title, cover_image, date, institution in zip(titles, cover_images, dates, institution_spans)
]



In [39]:
import json

In [40]:
# JSON 파일로 저장
with open('programs.json', 'w', encoding='utf-8') as f:
    json.dump(programs, f, ensure_ascii=False, indent=4)